This code solves arbitrary systems of linear equations using adiabatic quantum computing concepts.

References:

[1] https://arxiv.org/pdf/1805.10549

[2] https://arxiv.org/pdf/quant-ph/0001106

In [49]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
from qiskit.circuit.library import UnitaryGate
from qiskit.quantum_info import Operator
from qiskit.visualization import plot_histogram
import numpy as np
import math
import time

from ProblemData import ProblemData

In [58]:
def check_matrix_format(A_matrix):
   # Extract the matrix from the Operator
    matrix = A_matrix.to_matrix()

    # Check if matrix is square
    assert matrix.shape[0] == matrix.shape[1], "Matrix must be square."

    # Check if matrix is unitary
    identity_matrix = np.eye(matrix.shape[0])
    assert np.allclose(identity_matrix, matrix @ matrix.T.conj()), "Matrix must be unitary."


Return the A(s) operator from equation 3 of [1]

In [55]:
def get_A_s(s, A):
    n_qubits = int(np.log2(len(A)))

    # Define Pauli matrices as NumPy arrays
    Z = Operator(np.array([[1, 0], [0, -1]]))
    X = Operator(np.array([[0, 1], [1, 0]]))
    I_n = Operator(np.eye(2 ** n_qubits))

    # Compute tensor products
    Z_tensor = Z.tensor(I_n)
    X_tensor = X.tensor(Operator(A))

    # Construct the A_s matrix
    A_s = (1 - s) * Z_tensor + s * X_tensor

    check_matrix_format(A_s)
    
    return A_s

Adiabatic solver function

In [56]:
def adiabatic_solver(A_matrix, b_vec, T, M, plot_evolution=False, verbose=False):
    # Normalize
    A_matrix = A_matrix / np.linalg.norm(A_matrix)
    b_vec = b_vec / np.linalg.norm(b_vec)

    # Initialize the quantum circuit
    n_qubits = int(np.log2(len(A_matrix))) + 1
    qc = QuantumCircuit(n_qubits)

    # Define the initial state |psi>
    '''set the initial state to be a "-" qubit and the b vector state,
    which is the eigenvector corresponding to the 0-eigenvalue of H(s) from equation 3 of [1]'''
    qc.initialize(np.kron([1/np.sqrt(2), -1/np.sqrt(2)], b_vec), qc.qubits)
    
    # Define the time step
    dt = T / M

    for l in range(M):
        s = l / M
        A_s = get_A_s(s, A_matrix)
        
        # Convert A_s to a UnitaryGate and apply to the circuit
        U_s = UnitaryGate(A_s)
        qc.append(U_s, qc.qubits)

    check_matrix_format(A_s)
    
    # Simulate the circuit
    simulator = Aer.get_backend('aer_simulator')
    qc.save_statevector()
    compiled_circuit = transpile(qc, simulator)
    result = simulator.run(compiled_circuit).result()
    final_state = result.get_statevector(qc)

    # Plotting the results
    if plot_evolution:
        plot_histogram(final_state.probabilities_dict())

    return final_state
    

Use PrpblemData class to create a linear system corresponding to a neutron transport equation discretization

In [59]:
data = ProblemData("input.txt")

# Create the vectors holding the material data at each discretized point
data.read_input("input.txt")
data.initialize_BC()
data.initialize_XSs() 

# make A matrix and b vector
if data.sim_method == "sp3":
    A_mat_size = 2 * (data.n_x) * (data.n_y)
    A_matrix, b_vec = data.sp3_construct_A_matrix(A_mat_size) 
elif data.sim_method == "diffusion":
    A_mat_size = (data.n_x) * (data.n_y)
    A_matrix, b_vec = data.diffusion_construct_A_matrix(A_mat_size)

# Input which T and M values to test
T_vec = [1000000]
M_vec = [1000]
n_bits = 1 + int(math.log2(len(A_matrix)))

# Real answer to linear system
print("real answer (scaled):")
real_psi_solution = np.linalg.inv(A_matrix).dot(b_vec)
real_psi_solution = real_psi_solution/np.linalg.norm(real_psi_solution)
print(real_psi_solution)

# Parametric solutions, run solver for many M and T values
psi_solutions = np.zeros((len(T_vec), len(M_vec), len(A_matrix)), dtype=np.complex128)
psi_error = np.zeros((len(T_vec), len(M_vec), len(A_matrix)), dtype=np.complex128)
time1 = time.perf_counter()
for i, T in enumerate(T_vec):
    for j, M in enumerate(M_vec):
        psi = adiabatic_solver(A_matrix, b_vec, T, M, plot_evolution=False, verbose=False)
        psi = psi[0:int(len(psi)/2)]
        psi = psi / np.linalg.norm(psi)
        psi_solutions[i,j,:] = psi
        psi_error[i,j,:] = psi - real_psi_solution

        print("T: ", T)
        print("M: ", M)
        print("psi: ", psi)
        print("psi absolute value: ", np.abs(psi))
        print("real psi: ", real_psi_solution)
        print("psi error: ", psi - real_psi_solution)
time2 = time.perf_counter()
print("solver run time: ", time2 - time1)


# plot parametric results
psi = psi.reshape((data.n_x, data.n_y))

min_val = min(np.min(np.abs(psi)), np.min(real_psi_solution))
max_val = min(np.max(np.abs(psi)), np.max(real_psi_solution))

ax = sns.heatmap(np.abs(psi), linewidth=0.5, cmap="jet", vmin=min_val, vmax=max_val)
plt.title("Quantum Solution")
plt.savefig('q_sol.png')
plt.figure()

real_psi_solution = real_psi_solution.reshape((data.n_x, data.n_y))
ax = sns.heatmap(real_psi_solution, linewidth=0.5, cmap="jet", vmin=min_val, vmax=max_val)
plt.title("Real Solution")
plt.savefig('real_sol.png')
plt.figure()
plt.show()

real answer (scaled):
[0.07268108 0.09791757 0.1030929  0.08964646 0.08964646 0.1030929
 0.09791757 0.07268108 0.09791757 0.16421758 0.17039745 0.11853306
 0.11853306 0.17039745 0.16421758 0.09791757 0.1030929  0.17039745
 0.17705097 0.12518853 0.12518853 0.17705097 0.17039745 0.1030929
 0.08964646 0.11853306 0.12518853 0.11127869 0.11127869 0.12518853
 0.11853306 0.08964646 0.08964646 0.11853306 0.12518853 0.11127869
 0.11127869 0.12518853 0.11853306 0.08964646 0.1030929  0.17039745
 0.17705097 0.12518853 0.12518853 0.17705097 0.17039745 0.1030929
 0.09791757 0.16421758 0.17039745 0.11853306 0.11853306 0.17039745
 0.16421758 0.09791757 0.07268108 0.09791757 0.1030929  0.08964646
 0.08964646 0.1030929  0.09791757 0.07268108]


AssertionError: Matrix must be unitary.